In [2]:
import requests
import time
from datetime import timedelta
from datetime import datetime
import pandas as pd
import urllib.request
from tqdm import tqdm
import json
import asyncio
import numpy as np
import os
from nltk import sent_tokenize
from nltk import word_tokenize

In [28]:
def scrape_and_save_data(pageno):
    city = []
    state = []
    date = []
    lccn = []
    text = []
    title = []

    try:
        # https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=white+man+ticket&format=json&page=1
        #https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=oberholtzer&rows=50&format=json
        r = s.get(f'https://chroniclingamerica.loc.gov/search/pages/results/?state=&date1=1770&date2=1963&proxtext=oberholtzer&rows=100&format=json&page={pageno}', timeout=20)
        r.raise_for_status()  # Check if the request was successful
        data = r.json()

        for item in data.get('items', []):
            city.append(item.get('city', np.nan))
            date.append(item.get('date', np.nan))
            state.append(item.get('state', np.nan))
            lccn.append(item.get('lccn', np.nan))
            text.append(item.get('ocr_eng', np.nan))
            title.append(item.get('title', np.nan))

        df = pd.DataFrame({
            'city': city,
            'state': state,
            'date': date,
            'lCCN': lccn,
            'text': text,
            'title': title
        })

        output_dir =  '/Users/mstudio/Library/CloudStorage/Box-Box/oberholtzer/raw-data/'
        os.makedirs(output_dir, exist_ok=True)
        df.to_csv(f'{output_dir}oberholtzer_{pageno}.csv', index=False)
        time.sleep(10)
    except requests.exceptions.RequestException as e:
        # print(f"Error: {e}")
        failed_page.append(pageno)

failed_page = []
s = requests.Session()

for pageno in tqdm(range(int(np.round(4590 / 100)) + 1)): #4590
    no = pageno+1
    if str(no) in [i.strip('oberholtzer_').strip('.csv') for i in os.listdir('/Users/mstudio/Library/CloudStorage/Box-Box/oberholtzer/raw-data/')]:
        continue
    else:
        scrape_and_save_data(no)


100%|██████████| 47/47 [01:55<00:00,  2.45s/it]


In [3]:
oberholtzer_path='/Users/mstudio/Library/CloudStorage/Box-Box/ticket/raw-data/'
concat_list=[]
for i in os.listdir(oberholtzer_path):
    if i.startswith('.'):
        continue
    else:
        df = pd.read_csv(oberholtzer_path+i)
        df['filename'] = i.replace('.csv', '')  # Add new column with filename
        concat_list.append(df)

In [15]:
ticket_df=pd.concat(concat_list)
ticket_df.to_excel('/Users/mstudio/Library/CloudStorage/Box-Box/oberholtzer/oberholtzer.xlsx', index=False)
